In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as numpy
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
# from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics

In [3]:
data = pd.read_csv("Covid Data.csv")

In [5]:
data.tail(10)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
1048565,1,13,1,1,9999-99-99,97,2,39,2,2,...,2,2,2,2,2,2,2,2,7,97
1048566,2,13,2,1,9999-99-99,97,2,24,97,2,...,1,2,2,2,2,2,2,2,7,97
1048567,1,13,1,2,9999-99-99,2,2,23,2,1,...,2,2,2,2,2,1,2,2,7,2
1048568,1,13,2,1,9999-99-99,97,2,47,97,1,...,2,2,2,2,2,2,2,2,7,97
1048569,1,13,1,2,9999-99-99,2,2,56,2,1,...,2,2,2,2,2,2,2,2,7,2
1048570,2,13,2,1,9999-99-99,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,9999-99-99,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,9999-99-99,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,9999-99-99,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97
1048574,2,13,2,1,9999-99-99,97,2,52,97,2,...,2,2,2,2,2,2,2,2,7,97


In [10]:
def count_missing_vals(df_1):
    cols = df_1.columns
    missing_vals_count = {}
    for col in cols:
        if col != 'AGE':
            missing_vals_count[col] = sum(df_1[col]==97)
            if col == "DATE_DIED":
                missing_vals_count[col] = sum(df_1[col]=='9999-99-99')
    return missing_vals_count

In [11]:
missing_vals_count = count_missing_vals(data)
missing_vals_count

{'USMER': 0,
 'MEDICAL_UNIT': 0,
 'SEX': 0,
 'PATIENT_TYPE': 0,
 'DATE_DIED': 971633,
 'INTUBED': 848544,
 'PNEUMONIA': 0,
 'PREGNANT': 523511,
 'DIABETES': 0,
 'COPD': 0,
 'ASTHMA': 0,
 'INMSUPR': 0,
 'HIPERTENSION': 0,
 'OTHER_DISEASE': 0,
 'CARDIOVASCULAR': 0,
 'OBESITY': 0,
 'RENAL_CHRONIC': 0,
 'TOBACCO': 0,
 'CLASIFFICATION_FINAL': 0,
 'ICU': 848544}

In [14]:
preg_missing_vals = missing_vals_count['DATE_DIED']/data.shape[0]
print(preg_missing_vals)

0.9266223207686622


In [17]:
df = data.drop(['PREGNANT','DATE_DIED','INTUBED','ICU'], axis=1)
df.head(10)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,PNEUMONIA,AGE,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL
0,2,1,1,1,1,65,2,2,2,2,1,2,2,2,2,2,3
1,2,1,2,1,1,72,2,2,2,2,1,2,2,1,1,2,5
2,2,1,2,2,2,55,1,2,2,2,2,2,2,2,2,2,3
3,2,1,1,1,2,53,2,2,2,2,2,2,2,2,2,2,7
4,2,1,2,1,2,68,1,2,2,2,1,2,2,2,2,2,3
5,2,1,1,2,1,40,2,2,2,2,2,2,2,2,2,2,3
6,2,1,1,1,2,64,2,2,2,2,2,2,2,2,2,2,3
7,2,1,1,1,1,64,1,2,2,1,1,2,2,2,1,2,3
8,2,1,1,2,2,37,1,2,2,2,1,2,2,1,2,2,3
9,2,1,1,2,2,25,2,2,2,2,2,2,2,2,2,2,3


In [38]:
df["CLASIFFICATION_FINAL"].unique()

array([3, 5, 7, 6, 1, 2, 4], dtype=int64)

In [42]:
#Date_died: 9999-99-99: the patient is still alive else otherwise
def Death(x):
    if x == '9999-99-99':
        return 0
    else:
        return 1 
df['DIED'] = df['DATE_DIED'].apply(Death)
df.drop(columns=['DATE_DIED'], inplace=True)

In [46]:
df['Covid'] = [0 if i>=4 else 1 for i in df.CLASIFFICATION_FINAL]
df.drop(columns=['CLASIFFICATION_FINAL'], inplace=True)

In [56]:
missing_vals_count = count_missing_vals(df)
missing_vals_count

{'USMER': 0,
 'MEDICAL_UNIT': 0,
 'SEX': 0,
 'PATIENT_TYPE': 0,
 'PNEUMONIA': 0,
 'DIABETES': 0,
 'COPD': 0,
 'ASTHMA': 0,
 'INMSUPR': 0,
 'HIPERTENSION': 0,
 'OTHER_DISEASE': 0,
 'CARDIOVASCULAR': 0,
 'OBESITY': 0,
 'RENAL_CHRONIC': 0,
 'TOBACCO': 0,
 'DIED': 0,
 'Covid': 0}

# Training of the machine learning models

In [60]:
X_df = df.drop(columns=['Covid'], axis=1)
Y_class = df['Covid']

# Splitting data in train and test sets

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X_df,Y_class, test_size=0.30, random_state = 1,stratify=Y_class)

In [83]:
lr = LogisticRegression(solver='newton-cg',random_state=1,fit_intercept=False,class_weight={0:0.15,1:0.85})
model  = lr.fit(X_train,y_train)

In [88]:
train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train,train_pred)

In [89]:
train_acc

0.38903708709240575